In [ ]:
using System.Net;
using System.Net.Http;
using System.Net.NetworkInformation;

In [ ]:
public class BinanceApp
{
    private const string BASE_URL = "https://api.binance.com";

    private bool isRunning = true;

    private HttpClient httpClient = null;
    private BinancePingService pingService = null;

    public BinanceApp()
    {
        this.httpClient = new HttpClient();

        this.pingService = new BinancePingService(this.httpClient, BASE_URL, "/api/v3/ping");
    }

    public void StartApp()
    {
        if (!this.TryPingBaseUrl() || !this.TryApiPingRequest()) {
            Console.WriteLine("Failed to ping app.");
            return;
        }

        Console.WriteLine("Ping app successfully");

        this.isRunning = true;
    }

    public void UpdateApp()
    {

    }

    /// Ping ap ping path to check if the app is running.
    private bool TryApiPingRequest()
    {
        Task<bool> updateTask = Task.Run<bool>(async () => await this.pingService.TryApiPingRequest());
        return updateTask.Result;
    }

    /// Ping the base url to check if the app is running.
    private bool TryPingBaseUrl()
    {
        Task<bool> updateTask = Task.Run<bool>(async () => await this.pingService.TryPingBaseUrl());
        return updateTask.Result;
    }
}

public class BinancePingService
{
    private const string API_PING_METHOD = "GET";

    private string apiBaseUrl = string.Empty;
    private string apiPingPath = string.Empty;

    private HttpClient httpClient = null;

    public BinancePingService(HttpClient httpClient, string apiBaseUrl, string apiPingPath)
    {
        this.httpClient = httpClient;
        this.apiBaseUrl = apiBaseUrl;
        this.apiPingPath = apiPingPath;
    }

    public async Task<bool> TryApiPingRequest()
    {
        try {
            string endPoint = $"{this.apiBaseUrl}{this.apiPingPath}";
            HttpResponseMessage apiPingresult = await this.httpClient.GetAsync(endPoint);

            HttpResponseMessageHandler.LogResponse((apiPingresult));
            
            return apiPingresult.StatusCode == HttpStatusCode.OK;
        }
        catch (Exception e) {
            ExceptionHandler.LogException(e);
        }

        return false;
    }

    public async Task<bool> TryPingBaseUrl()
    {
        try {
            Ping ping = new Ping();
            // Ping url must not contain {https://}.
            PingReply pingResult = await ping.SendPingAsync(this.apiBaseUrl.Replace("https://", ""));

            PingReplyHandler.LogReply(pingResult);
            
            return pingResult.Status == IPStatus.Success;
        } catch (Exception e) {
            ExceptionHandler.LogException(e);
        }

        return false;
    }
}

public static class ExceptionHandler
{
    private static StringBuilder messageBuilder = new StringBuilder();

    public static void LogException(Exception e)
    {
        messageBuilder.AppendLine($"___MESSAGE:{e.Message}.");
        messageBuilder.AppendLine($"___STACKTRACE:{e.StackTrace}.");
        
        Console.WriteLine(messageBuilder.ToString());
    }
}

public static class HttpResponseMessageHandler
{
    private static StringBuilder messageBuilder = new StringBuilder();

    public static async void LogResponse(HttpResponseMessage response)
    {
        string result = await response.Content.ReadAsStringAsync();

        messageBuilder.AppendLine($"___STATUS:{response.StatusCode}.");
        messageBuilder.AppendLine($"___CONTENT:{result}.");

        Console.WriteLine(messageBuilder.ToString());
    }
}

public static class PingReplyHandler
{
    private static StringBuilder messageBuilder = new StringBuilder();

    public static void LogReply(PingReply reply)
    {
        messageBuilder.AppendLine($"__ADDRESS: {reply.Address}");
        messageBuilder.AppendLine($"ROUNTRIP TIME: {reply.RoundtripTime}");
        messageBuilder.AppendLine($"__BUFFER: {reply.Buffer.Length}");
        messageBuilder.AppendLine($"__STATUS: {reply.Status}");

        Console.WriteLine(messageBuilder.ToString());
    }
}

BinanceApp app = new BinanceApp();
app.StartApp();

__ADDRESS: 13.227.6.143
ROUNTRIP TIME: 8
__BUFFER: 32
__STATUS: Success

___STATUS:OK.
___CONTENT:{}.

Ping app successfully
